In [1]:
import pandas as pd
import data_util
from tqdm.notebook import tqdm
#from tqdm import tqdm_notebook as tqdm
from data_generator import DataGenerator
from state_util import StateUtil
from tec_an import TecAn
import numpy as np
from data_util import *
from sklearn_model_hyper import *
from joblib import dump, load
from data_generator import DataGenerator
from data_agent import DataAgent, TacProcess
from stock_agent import *
from sklearn.utils import shuffle

import math
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as scs
import scikitplot as skplt

from keras.wrappers.scikit_learn import KerasClassifier
from imblearn.over_sampling import RandomOverSampler
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
import tensorflow.keras as keras
import random
from catboost import CatBoost

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, Normalizer
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.metrics import precision_score, recall_score, f1_score


from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV

path = "./data/"


In [2]:
def get_y_data(ohlc, shift = -1):
    combined_data = ohlc.copy()
    #combined_data['return'] = np.log(combined_data / combined_data.shift(1))
    returns = (ohlc / ohlc.shift(shift))
    combined_data['return'] = returns
    combined_data['direction'] = np.where(combined_data['return'] < 1, 1, 0)
    #print(combined_data)
    #combined_data.dropna(inplace=True)
    #print(combined_data[20:40])
    #
    return combined_data['direction'].to_numpy()


def get_sequencial_data(trainX, trainY, step):
    y = get_y_data(
        pd.DataFrame(trainY, columns = ['Close']), 
        (-1 * step)
    )
    x, y, closed_prices = trainX, y, trainY
    return x, y, closed_prices

def prepare_train_data(trainX, trainY, step):
    y = get_y_data(
        pd.DataFrame(trainY, columns = ['Close']), 
        (-1 * step)
    )
    x, y, closed_prices = trainX, y, trainY
    return shuffle(x, y, closed_prices, random_state = 10)

In [3]:
def avg(array):
    if (len(array) == 0):
        return 0
    return sum(array)/len(array)

In [4]:
%%time

classifiers = [
    LogisticRegression(),
    KNeighborsClassifier(3),
    #GaussianProcessClassifier(1.0 * RBF(1.0)),
    DecisionTreeClassifier(),
    RandomForestClassifier(max_depth=50, n_estimators=100, max_features=1),
    MLPClassifier(alpha=1, max_iter=1000),
    AdaBoostClassifier(),
    CatBoostClassifier(logging_level = 'Silent'),
    SVC(kernel="linear", C=0.025),
    SVC(gamma=2, C=1),
    GaussianNB(),
    QuadraticDiscriminantAnalysis()
]


CPU times: user 237 µs, sys: 24 µs, total: 261 µs
Wall time: 264 µs


In [5]:
def backtest_model(model, x, y, closed_prices):
    back = BackTest(value = 100, 
                    verbose = False, 
                    sell_on_profit = True,
                    pending_sell_steps = step)
    
    for idx in range(len(x)):
        xx = [x[idx]]
        yy = model.predict(xx)[0]
        price = closed_prices[idx]
        #print(f'{idx} {yy} {price}')
        back.on_state(0, price)
        if(yy == 1):
            back.request_buy(price)
        else:
            back.request_sell(price)
    return back

In [24]:
normalizer = tf.keras.layers.experimental.preprocessing.Normalization()
#normalizer = tf.keras.layers.experimental.preprocessing.Normalization(input_shape=(features,))


def train_by_step(model, step):
    trainX_raw, trainY_raw = load_data("simple_full_", "train", path)
    x, y, closed_prices = prepare_train_data(trainX_raw, trainY_raw, step)
    #print(f"{trainX_raw.shape}")
    #print(closed_prices)
    model.fit(x, y)

def eval_step(model, train_set, step):

    valX, valY = load_data(f"simple_{train_set}", "train", path)
    
    x, y, closed_prices = get_sequencial_data(valX, valY, step)

    recall = recall_score(y, model.predict(x))
    precision = precision_score(y, model.predict(x))
    f1 = f1_score(y, model.predict(x))
    
    back = backtest_model(model, x, y, closed_prices)
    return back, (precision, recall, f1)

    
param = {'criterion': 'entropy', 'max_depth': 100, 'n_estimators': 30, 'random_state': 42}
model = make_pipeline(StandardScaler(), RandomForestClassifier(**param))
model = RandomForestClassifier(**param)


param = {'depth': 10, 'eval_metric': 'Precision', 'iterations': 500, 'l2_leaf_reg': 1e-19, 'logging_level': 'Silent', 'random_seed': 42}
model = make_pipeline(StandardScaler(),CatBoostClassifier(**param))


In [25]:
class MockModel():
    
    def __init__(self, 
                 model
                ):
        self.model = model
        self.normalizer = tf.keras.layers.experimental.preprocessing.Normalization()
        
        
    def fit(self, x, y):
        self.normalizer.adapt(x)
        norm_x = self.normalizer(x).numpy()
        self.model.fit(norm_x, y)
        
    def predict(self, x):
        norm_x = self.normalizer(x).numpy()
        return self.model.predict(norm_x)
        
model = MockModel(model)

In [26]:
def get_max_profit(x, y, closed_prices, step):

    back = BackTest(value = 100, 
                    verbose = False, 
                    sell_on_profit = True,
                    pending_sell_steps = step)
    
    for idx in range(len(x)):
        yy = y[idx]
        price = closed_prices[idx]
        #print(f'{idx} {yy} {price}')
        back.on_state(0, price)
        #print(yy)
        if(yy == 1):
            back.request_buy(price)
        else:
            back.request_sell(price)
    return back

In [ ]:
%%time
trains_sets = ["btcusd17", "backtest", "4jul21"]
#trains_sets = ["backtest"]
#trains_sets = ["backtest", "4jul21"]
dirs = ["omgusd", "btceur", "btcusd17", "btcusd", "ethusd", "ltcusd", "btcusdAug19"]

#trains_sets = dirs

reference_profit = {}
models_profit = {}

models_profit_metric = {}

models_score = {}

keys = []

steps = 5

for train_set in trains_sets:
    trainX_raw, trainY_raw = load_data(f"simple_{train_set}", "train", path)
    print(f"Dataset: {train_set}")
    for step in tqdm(range(1, steps + 1)):
        x, y, closed_prices = get_sequencial_data(trainX_raw, trainY_raw, step)
        reference = get_max_profit(x, y, closed_prices, step)
        
        key = f"{train_set}_{step}"
        keys.append(key)
        
        reference_profit[key] = reference.get_profit()
        
        back, score = eval_step(model, train_set, step)
        #models_profit[key] = f"{back.get_profit()}"
        models_profit[key] = back.get_profit()
        models_score[key] = score
        models_profit_metric[key] = back.get_profit() / reference.get_profit()


Dataset: btcusd17


  0%|          | 0/5 [00:00<?, ?it/s]

In [ ]:
# Cat

for key in keys:
    print(f"{key}: {reference_profit[key]} | {models_profit_metric[key]} | {models_profit[key]} {models_score[key]}")


In [18]:
# Florest

for key in keys:
    print(f"{key}: {reference_profit[key]} | {models_profit[key]} {models_score[key]}")


btcusd17_1: 1020.3814 | 919.6024 (0.9996334982591167, 0.9990842490842491, 0.9993587982046349)
btcusd17_2: 778.9804 | 678.9804 (0.9996580027359782, 1.0, 0.9998289721224559)
btcusd17_3: 665.1563 | 565.7937 (0.999322951929587, 1.0, 0.9996613613274636)
btcusd17_4: 599.8734 | 499.1493 (0.9989905787348586, 0.9998316214850985, 0.9994109231675503)
btcusd17_5: 542.1894 | 442.1892 (0.9991666666666666, 0.9998332221480988, 0.9994998332777594)
backtest_1: 135.1525 | 1.9994 (0.48417350527549824, 0.4134134134134134, 0.44600431965442766)
backtest_2: 103.3888 | -100.3124 (0.5079207920792079, 0.5109561752988048, 0.5094339622641509)
backtest_3: 85.2341 | -93.2661 (0.49129989764585463, 0.47244094488188976, 0.48168590065228295)
backtest_4: 74.232 | -104.5745 (0.4953271028037383, 0.4622093023255814, 0.4781954887218045)
backtest_5: 66.139 | -99.6539 (0.4852492370295015, 0.4658203125, 0.47533632286995514)
4jul21_1: 11.0982 | 0.0973 (0.5133333333333333, 0.5460992907801419, 0.529209621993127)
4jul21_2: 7.7523 |

In [ ]:
back = BackTest(value = 100, 
                verbose = True, 
                sell_on_profit = False,
                pending_sell_steps = step)

In [ ]:
back.on_state(0, 0)
back.request_buy(100)

In [ ]:
back.on_state(0, 0)
back.request_sell(110)

In [ ]:
back.current

In [ ]:
bitcoin = 50000
money = 100000


final = money / bitcoin

print(final)

In [ ]:
bitcoin = 1
money = 100


final = bitcoin * money 

print(final)